# 加载Zhou数据并分区

In [3]:
import numpy as np
import pandas as pd
import os, sys
sys.path.append("..")


from tools.processing import pd_read_dbf

from matplotlib import pyplot as plt
%matplotlib inline

## PNAS数据

In [4]:
pns = pd.read_excel(io='../data/Zhou et al_2020_PNAS_dataset.xlsx', sheet_name='D1')

# 清洗数据的头部，让两行头部变一行
change_name_dic = {}
last_item = "None"
for col in pns:
    second_row = pns.loc[0, col]
    if "Unnamed" in col:
        change_name_dic[col] = last_item + ": " + second_row
    else:
        if type(second_row) is str:
            change_name_dic[col] = col + ": " + second_row
        last_item = col

pns.rename(change_name_dic, axis=1, inplace=True)
pns = pns.drop(0)

# 重命名表头，取消两边的空格
pns.rename({col: col.strip() for col in pns}, axis=1, inplace=True)

# 更改正确的数据类型
pns = pns.astype(float, errors='ignore')
pns['Year'] = pns['Year'].astype(int)
pns.iloc[:, 2:] = pns.iloc[:, 2:].astype(float)

pns.head()

,City_ID,Year,IRR,Irrigated area: Total,Irrigated area: Rice,Irrigated area: Wheat,Irrigated area: Maize,Irrigated area: Vegetables and fruits,Irrigated area: Others,Irrigation water-use intensity (WUI): Total,...,Service GVA,Service WUI,RUR,Rural domestic WU,Rural population,Rural domestic WUI,Rural livestock WU,Livestock population,Livestock WUI,Total water use
1,C1,1965,1.229927,127.315625,59.834838,8.289719,0.443002,2.697220,56.050846,966.045314,...,0.044986,0.016339,0.016483,0.014680,0.469140,85.727127,0.001804,110.158490,0.016373,1.261649
2,C1,1966,1.274452,129.968097,62.355186,7.819602,0.404165,2.741800,56.647345,980.588287,...,0.053715,0.015736,0.018686,0.016790,0.516349,89.085128,0.001896,115.778437,0.016380,1.308981
3,C1,1967,1.482867,152.163380,68.736923,9.776923,0.489071,3.267321,69.893142,974.522696,...,0.062626,0.014725,0.020151,0.018158,0.561759,88.556282,0.001993,121.665631,0.016385,1.519409
4,C1,1968,1.538731,156.787722,74.005815,10.511051,0.529418,3.443728,68.297710,981.410519,...,0.072619,0.013645,0.021343,0.019248,0.605370,87.108619,0.002095,127.834367,0.016389,1.577010
5,C1,1969,1.642901,165.174460,83.210254,10.657419,0.569342,3.536641,67.200804,994.645761,...,0.088720,0.012364,0.023139,0.020938,0.647184,88.636981,0.002201,134.298872,0.016391,1.683614


## 黄河流域相交的市县

In [7]:
# 所有与黄河流域相交的市县

fn = r'data/cities_shapefile_zhou2020/perfectures_YR.dbf'
yr = pd_read_dbf(fn)
yr = yr.set_index(yr['Perfecture'], drop=True).drop('Perfecture', axis=1)
yr.head()

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 0: ordinal not in range(128)

In [8]:
# 根据相交的市县 ID， 在PNAS表中提取数据

city_yr = pns[pns.City_ID.isin(yr.index.unique())]
city_yr.shape
len(city_yr['City_ID'].unique())
city_yr.head()

(3773, 55)

77

,City_ID,Year,IRR,Irrigated area: Total,Irrigated area: Rice,Irrigated area: Wheat,Irrigated area: Maize,Irrigated area: Vegetables and fruits,Irrigated area: Others,Irrigation water-use intensity (WUI): Total,...,Service GVA,Service WUI,RUR,Rural domestic WU,Rural population,Rural domestic WUI,Rural livestock WU,Livestock population,Livestock WUI,Total water use
1275,C27,1965,0.300518,46.631997,0.391448,16.089679,1.152312,0.571298,28.427261,644.445209,...,0.024149,0.077970,0.009021,0.005818,0.499787,31.895556,0.003203,141.750766,0.022595,0.328586
1276,C27,1966,0.323595,49.468303,0.383836,16.485679,1.434736,0.636613,30.527440,654.146772,...,0.036026,0.051291,0.009069,0.005733,0.553613,28.371723,0.003336,147.646616,0.022592,0.351996
1277,C27,1967,0.340063,52.309331,0.416675,17.803304,1.442818,0.697033,31.949501,650.100439,...,0.028569,0.059049,0.008063,0.004650,0.605625,21.033715,0.003413,151.033245,0.022600,0.372432
1278,C27,1968,0.352690,53.870788,0.437429,18.863369,1.514685,0.770592,32.284713,654.695280,...,0.027334,0.066526,0.008809,0.005322,0.655823,22.233352,0.003487,154.266500,0.022604,0.391458
1279,C27,1969,0.365740,55.120730,0.447621,19.700679,1.549920,0.752680,32.669830,663.524461,...,0.032263,0.071070,0.012013,0.008437,0.704206,32.825618,0.003575,158.251492,0.022594,0.406136


In [9]:
# 检查和之前做的是否一致
old_vision = pd.read_csv('data/YR_perfectures/perfectures_in_YRB.csv', index_col=0)
old_vision.shape
len(old_vision['City_ID'].unique())

(3773, 57)

77

In [10]:
# 旧版本（使用省份进行分区的结果）：
old_vision['Region'].value_counts()

DR    1127
UR    1127
MR    1029
SR     392
Name: Region, dtype: int64

## 判断分区

- 分别加载源区、上游、中游、下游
- 对每一个与黄河流域相交的市，判断其与四个区域中的哪个相交
- 如果相交，则提取相交部分的面积
- 对单一相交的，进行区域标记，记录面积为其相交面积
- 对重复相交的，判断哪个相交部分面积更大，进行区域标记，以该面积为相交面积
- 对标记好的区域，按照相交部分面积，计算修正比例系数

In [11]:
# 分别加载每个区域的相交市县
regions = ['SR', 'UR', 'MR', 'DR']

# 对每个市县计算其与各个流域相交部分的面积
intersect_area = {region: {} for region in regions}
for region in regions:
    file_path = r'data/cities_shapefile_zhou2020/{}_Intersect.dbf'.format(region)
    df = pd_read_dbf(file_path)
    area_index = df.columns.tolist().index('Area_calcu')  # 面积的索引
    city_index = df.columns.tolist().index('Perfecture')  # 市ID的索引
    for row in df.itertuples(index=False):
        city = row[city_index]
        area = row[area_index]
        intersect_area[region][city] = area
        
perfectures = pd.DataFrame(intersect_area).fillna(0.)
perfectures.head()

,SR,UR,MR,DR
C270,17044.939507,0.670980,0.000000,0.0
C29,9761.387069,19312.020379,15.133872,0.0
C275,1692.224543,0.000000,0.000000,0.0
C223,57666.892719,0.000000,0.000000,0.0
C226,18334.619415,9224.416286,0.000000,0.0


In [12]:
# 对每个市县，比较其在每个区域的面积，取其中最大的

for row in perfectures.itertuples():
    city_id = row[0]
    area_tuple = row[1:]
    max_area = max(area_tuple)
    region = perfectures.columns[area_tuple.index(max_area)]
    yr.loc[city_id, 'Region'] = region
    yr.loc[city_id, 'Intersect_area'] = max_area

yr['Ratio'] = yr['Intersect_area'] / yr['Area_calcu']  # 计算最大面积占比
yr.head()

,Province_n,Area_calcu,Region,Intersect_area,Ratio
Perfecture,,,,,
C270,Sichuan,82968.009569,SR,17044.939507,0.205440
C206,Neimeng,239467.190080,UR,3588.328656,0.014985
C259,Shaanxi,23532.928063,MR,29.559255,0.001256
C97,Henan,7349.992243,DR,1717.139623,0.233625
C207,Neimeng,65134.010370,UR,33718.251040,0.517675


In [13]:
# 进行交互式展示
# 用来实现交互页面的包
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# 展示最大面积占比少于某个阈值的所有城市
@interact
def show_area_ratio_less_than(x=widgets.FloatSlider(min=0., max=.1, step=0.01, value=0.05)):
    data = yr.loc[yr['Ratio'] < x].sort_values(by=['Ratio', 'Region'], axis=0)
    return data

interactive(children=(FloatSlider(value=0.05, description='x', max=0.1, step=0.01), Output()), _dom_classes=('…

In [14]:
# 对照旧的版本，检查有多少城市的区域发生了变化

old_new_region = {}
for city, old_region in old_vision.groupby(['City_ID', 'Region']).count().index.tolist():
    new_region = yr.loc[city, 'Region']
    if old_region != new_region:
        old_new_region[city] = [old_region, new_region]
        
"There are {} ({:.2%}) citys corrected in new regions.".format(len(old_new_region), len(old_new_region)/len(yr))
old_new_region

'There are 18 (23.38%) citys corrected in new regions.'

{'C100': ['DR', 'MR'],
 'C102': ['DR', 'MR'],
 'C104': ['DR', 'MR'],
 'C105': ['DR', 'MR'],
 'C107': ['DR', 'MR'],
 'C112': ['DR', 'MR'],
 'C218': ['UR', 'MR'],
 'C224': ['SR', 'UR'],
 'C225': ['SR', 'UR'],
 'C227': ['SR', 'UR'],
 'C229': ['SR', 'UR'],
 'C268': ['MR', 'UR'],
 'C28': ['UR', 'MR'],
 'C35': ['UR', 'MR'],
 'C36': ['UR', 'MR'],
 'C37': ['UR', 'MR'],
 'C38': ['UR', 'MR'],
 'C99': ['DR', 'MR']}

## 存储数据

- 选择某个阈值以上的所有数据
- 存储数据

In [15]:
def get_data_with_threshold(threshold):
    filtered_yr = yr[yr['Ratio'] > threshold].copy()
    filtered_yr.drop('Province_n', axis=1, inplace=True)
    filtered_yr.reset_index(inplace=True)
    data = pns[pns.City_ID.isin(filtered_yr['Perfecture'].values)]
    return pd.merge(left=data, right=yr, left_on='City_ID', right_on='Perfecture')

get_data_with_threshold(0.05)

,City_ID,Year,IRR,Irrigated area: Total,Irrigated area: Rice,Irrigated area: Wheat,Irrigated area: Maize,Irrigated area: Vegetables and fruits,Irrigated area: Others,Irrigation water-use intensity (WUI): Total,...,Rural domestic WUI,Rural livestock WU,Livestock population,Livestock WUI,Total water use,Province_n,Area_calcu,Region,Intersect_area,Ratio
0,C27,1965,0.300518,46.631997,0.391448,16.089679,1.152312,0.571298,28.427261,644.445209,...,31.895556,0.003203,141.750766,0.022595,0.328586,Gansu,20091.467281,UR,19188.439369,0.955054
1,C27,1966,0.323595,49.468303,0.383836,16.485679,1.434736,0.636613,30.527440,654.146772,...,28.371723,0.003336,147.646616,0.022592,0.351996,Gansu,20091.467281,UR,19188.439369,0.955054
2,C27,1967,0.340063,52.309331,0.416675,17.803304,1.442818,0.697033,31.949501,650.100439,...,21.033715,0.003413,151.033245,0.022600,0.372432,Gansu,20091.467281,UR,19188.439369,0.955054
3,C27,1968,0.352690,53.870788,0.437429,18.863369,1.514685,0.770592,32.284713,654.695280,...,22.233352,0.003487,154.266500,0.022604,0.391458,Gansu,20091.467281,UR,19188.439369,0.955054
4,C27,1969,0.365740,55.120730,0.447621,19.700679,1.549920,0.752680,32.669830,663.524461,...,32.825618,0.003575,158.251492,0.022594,0.406136,Gansu,20091.467281,UR,19188.439369,0.955054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,C270,2009,0.049847,18.607523,0.000000,1.451464,4.767338,3.156115,9.232606,267.887520,...,16.517526,0.025332,1066.248649,0.023758,0.116826,Sichuan,82968.009569,SR,17044.939507,0.205440
2936,C270,2010,0.051362,18.657270,0.000000,1.434741,4.831198,3.249039,9.142292,275.294267,...,18.618851,0.024486,1033.055432,0.023702,0.125504,Sichuan,82968.009569,SR,17044.939507,0.205440
2937,C270,2011,0.052419,18.756611,0.000000,1.417535,4.824774,3.342370,9.171932,279.471819,...,20.789793,0.024262,1026.433290,0.023638,0.133171,Sichuan,82968.009569,SR,17044.939507,0.205440
2938,C270,2012,0.058520,19.203319,0.000000,1.405644,4.910646,3.494206,9.392824,304.741155,...,24.039309,0.023224,982.120733,0.023647,0.146506,Sichuan,82968.009569,SR,17044.939507,0.205440
